# DBAAS Hello World and Test Truncate

# Import to run %sh magic cell

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

# Add Java Client POM Dependency

In [2]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>io.netty</groupId>
    <artifactId>netty-transport</artifactId>
     <version>4.1.93.Final</version>
  </dependency>
   <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-proxy-client</artifactId>
    <version>7.0.0</version>
  </dependency>
</dependencies>

# Test Hello World!

In [3]:
import com.aerospike.client.proxy.AerospikeClientProxy;
import com.aerospike.client.policy.*;
import com.aerospike.client.*;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;

//Required Imports
//import com.aerospike.client.AerospikeClient;
//import com.aerospike.client.policy.WritePolicy;
//import com.aerospike.client.Bin;
//import com.aerospike.client.Key;
//import com.aerospike.client.Record;
//import com.aerospike.client.Value;
//import com.aerospike.client.Operation;
import com.aerospike.client.query.Statement;
import com.aerospike.client.exp.Exp;

final String AEROSPIKE_CLOUD_HOST="3d4f4845-08b9-4de6-b124-b902e2316903.aerospikedev.com";
final String AEROSPIKE_CLOUD_KEY="b9236be6659544c887dba7c42dfed842"; 
final String AEROSPIKE_CLOUD_SECRET="45cc6e15714a4b3f93077154d7b5375b";
final String AEROSPIKE_CLOUD_NAMESPACE="aerospike_cloud";
final int AEROSPIKE_CLOUD_PORT=4000;

ClientPolicy policy = new ClientPolicy();
policy.user = AEROSPIKE_CLOUD_KEY;
policy.password = AEROSPIKE_CLOUD_SECRET;

TlsPolicy tlsPolicy = new TlsPolicy();
policy.tlsPolicy = tlsPolicy;

AerospikeClientProxy client = new AerospikeClientProxy(policy, 
           new Host(AEROSPIKE_CLOUD_HOST, AEROSPIKE_CLOUD_PORT) 
                );

//Key key = new Key(AEROSPIKE_CLOUD_NAMESPACE, null, "my_key");  
//Check: Dbaas Allows writing records to null set but does not allow creating
//sindex on entire namespace. ==> should disallow creating records in null set.

Key key = new Key(AEROSPIKE_CLOUD_NAMESPACE, "my_set", "my_key");
Bin bin = new Bin("my_bin", "Hello World!");

client.put(null, key, bin);
Record record = client.get(null, key);
System.out.println(record);

//client.close();


(gen:1),(exp:0),(bins:(my_bin:Hello World!))


# Test Truncate ... 

In [4]:
//Insert a record using a class based construct
class userProfile {
  public void createProfile (String ns, String setName, String user, String name, int age, boolean bCheck) {
    Key userKey = new Key(ns, setName, user); 
    Bin bName = new Bin("name", name);
    Bin bAge = new Bin("age", age);
    WritePolicy wPolicy1 = new WritePolicy();
    wPolicy1.sendKey = true;
    client.put(wPolicy1, userKey, bName, bAge);
    if(bCheck){
      System.out.println(client.get(null, userKey));
    }   
  }
}

In [5]:
userProfile pf = new userProfile();
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset", "jack","Jack", 26, true);

(gen:1),(exp:0),(bins:(name:Jack),(age:26))


In [6]:
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset", "jill","Jill", 32, false);

In [7]:
Key userJill = new Key(AEROSPIKE_CLOUD_NAMESPACE, "testset", "jill"); 
System.out.println(client.get(null, userJill));

(gen:1),(exp:0),(bins:(name:Jill),(age:32))


In [8]:
class Truncate {
  public void truncate(String ns, String setName, int sinceUpdate_ms) {
    WritePolicy wPolicy = new WritePolicy();
    if(sinceUpdate_ms > 0){
      //Add "Since Last Update Time" based Expression Filter
      wPolicy.filterExp = Exp.build(Exp.gt(Exp.sinceUpdate(), Exp.val(sinceUpdate_ms)));
    }
    
    Operation[] ops = new Operation[] {
      Operation.delete()
    };
    Statement stmt = new Statement();
    stmt.setNamespace(ns);
    if(setName != null){
      stmt.setSetName(setName);
    }
    stmt.setOperations(ops);
    client.execute(wPolicy, stmt);
  }
}

In [9]:
Truncate tr = new Truncate();
tr.truncate(AEROSPIKE_CLOUD_NAMESPACE, "testset", 0);

In [10]:
System.out.println(client.get(null, userJill));

null


In [11]:
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset1", "jack","Jack", 26, false);
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset2", "jill","Jill", 32,false);
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset3", "pat","Pat", 50,false);


In [12]:
Key user1 = new Key(AEROSPIKE_CLOUD_NAMESPACE, "testset1", "jack"); 
Key user2 = new Key(AEROSPIKE_CLOUD_NAMESPACE, "testset2", "jill");
Key user3 = new Key(AEROSPIKE_CLOUD_NAMESPACE, "testset3", "pat"); 

System.out.println(client.get(null, user1));
System.out.println(client.get(null, user2));
System.out.println(client.get(null, user3));

(gen:1),(exp:0),(bins:(name:Jack),(age:26))
(gen:1),(exp:0),(bins:(name:Jill),(age:32))
(gen:1),(exp:0),(bins:(name:Pat),(age:50))


In [13]:
Truncate tr = new Truncate();
tr.truncate(AEROSPIKE_CLOUD_NAMESPACE, null, 10000);

In [14]:
System.out.println(client.get(null, user1));
System.out.println(client.get(null, user2));
System.out.println(client.get(null, user3));

(gen:1),(exp:0),(bins:(name:Jack),(age:26))
(gen:1),(exp:0),(bins:(name:Jill),(age:32))
(gen:1),(exp:0),(bins:(name:Pat),(age:50))


In [19]:
//Try again, after 10 seconds of creating the records
tr.truncate(AEROSPIKE_CLOUD_NAMESPACE, null, 10000);

In [20]:
System.out.println(client.get(null, user1));
System.out.println(client.get(null, user2));
System.out.println(client.get(null, user3));

null
null
null


### Secondary Index Testing

#### Generate Test Data

In [21]:
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset", "jacksapplekey1","Jack", 26, true);
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset", "jillgrapekey2","Jill", 20, true);
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset", "mangokey3","James", 38, true);
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset", "Jimkey4apple","Jim", 46, true);
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset", "JuliaGrapekey5","Julia", 62, true);
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset", "SallyMANGOkey6","Sally", 32, true);
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset", "SeanaPPlekey7","Sean", 24, true);
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset", "SamGRAPEkey8","Sam", 12, true);
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset", "Susankey9","Susan", 42, true);
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset", "SandraPeachkey0","Sandra", 34, true);

(gen:1),(exp:0),(bins:(name:Jack),(age:26))
(gen:1),(exp:0),(bins:(name:Jill),(age:20))
(gen:1),(exp:0),(bins:(name:James),(age:38))
(gen:1),(exp:0),(bins:(name:Jim),(age:46))
(gen:1),(exp:0),(bins:(name:Julia),(age:62))
(gen:1),(exp:0),(bins:(name:Sally),(age:32))
(gen:1),(exp:0),(bins:(name:Sean),(age:24))
(gen:1),(exp:0),(bins:(name:Sam),(age:12))
(gen:1),(exp:0),(bins:(name:Susan),(age:42))
(gen:1),(exp:0),(bins:(name:Sandra),(age:34))


#### Test Sindex, Type  NUMERIC with Filter Expression

In [22]:
//Needed imports
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.query.RecordSet;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.exp.Exp.Type;
import com.aerospike.client.query.RegexFlag;

//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("aerospike_cloud");
stmt.setSetName("testset");
stmt.setFilter(Filter.range("age", 20,60));
QueryPolicy qp = new QueryPolicy();

//String[] keys = {"key1", "key2", "key8"};
Exp exp = Exp.val(false);  //init to false
//for (int i=0; i<keys.length; i++)
//{
   //exp = Exp.or(exp, Exp.eq(Exp.key(Type.STRING), Exp.val(keys[i]))); 
   exp = Exp.regexCompare("^.*apple.*",RegexFlag.ICASE,Exp.key(Type.STRING)); 
//}
qp.filterExp = Exp.build(exp);

// Add Expression
/*
qp.filterExp = Exp.build(Exp.or( 
  Exp.eq(Exp.key(Type.STRING), Exp.val("key1")),   //Jack
  Exp.eq(Exp.key(Type.STRING), Exp.val("key2")),   //Jill
  Exp.eq(Exp.key(Type.STRING), Exp.val("key8"))    //Sam, age 12, SI age reject.
  )
  );

*/


RecordSet rs = client.query(qp, stmt);
while (rs.next()){
  Record r = rs.getRecord();
  Key k = rs.getKey();  
  System.out.println(r);
    System.out.println(k);
}

EvalException: Error 201,1,0,30000,0,5: Index not found

#### Test Sindex, Type STRING, with Filter Expression

In [23]:
//Run SI query
Statement stmt2 = new Statement();
stmt2.setNamespace("aerospike_cloud");
stmt2.setSetName("testset");
stmt2.setFilter(Filter.equal("name", "Jill"));
QueryPolicy qp = new QueryPolicy();

//String[] keys = {"key1", "key2", "key8"};
Exp exp = Exp.val(false);  //init to false
//for (int i=0; i<keys.length; i++)
//{
   //exp = Exp.or(exp, Exp.eq(Exp.key(Type.STRING), Exp.val(keys[i]))); 
   exp = Exp.regexCompare("^.*grape.*",RegexFlag.ICASE,Exp.key(Type.STRING)); 
//}
qp.filterExp = Exp.build(exp);

// Add Expression
/*
qp.filterExp = Exp.build(Exp.or( 
  Exp.eq(Exp.key(Type.STRING), Exp.val("key1")),   //Jack
  Exp.eq(Exp.key(Type.STRING), Exp.val("key2")),   //Jill
  Exp.eq(Exp.key(Type.STRING), Exp.val("key8"))    //Sam, age 12, SI age reject.
  )
  );
*/



RecordSet rs = client.query(qp, stmt2);
while (rs.next()){
  Record r = rs.getRecord();
  Key k = rs.getKey();  
  System.out.println(r);
    System.out.println(k);
}

EvalException: Error 201,1,0,30000,0,5: Index not found

#### Test Sindex, Collection Data Types, with CTX

#### Test Sindex, Collection Data Types, with CTX  - Setup and Generate Test Records

In [24]:
Truncate tr = new Truncate();
tr.truncate(AEROSPIKE_CLOUD_NAMESPACE, "testset", 0);

In [25]:
// import all needed modules
import java.util.ArrayList;
import java.util.Arrays;
import java.util.HashMap;
import java.util.List;
import java.util.Map;

import com.aerospike.client.AerospikeException;
import com.aerospike.client.Key;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.policy.ClientPolicy;
import com.aerospike.client.Record;
import com.aerospike.client.Operation;
import com.aerospike.client.Value;
import com.aerospike.client.Value.ListValue;
import com.aerospike.client.Value.MapValue;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.cdt.ListPolicy;
import com.aerospike.client.cdt.ListOrder;
import com.aerospike.client.cdt.ListWriteFlags;
import com.aerospike.client.cdt.MapOperation;
import com.aerospike.client.cdt.MapPolicy;
import com.aerospike.client.cdt.MapOrder;
import com.aerospike.client.cdt.MapWriteFlags;
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.query.RecordSet;
import com.aerospike.client.Record;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.query.IndexType;
import com.aerospike.client.query.IndexCollectionType;
import com.aerospike.client.task.IndexTask;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;
import com.aerospike.client.cdt.CTX;

In [26]:
final String Namespace = "aerospike_cloud";
final String Set = "testset";
final String ListBin = "list_bin";
final String MapBin = "map_bin";

## CDT Test Data
We will illustrate how to create indexes on CDTs and issues queries using two CDTs - a nested List and a nested Map that are populated in 10 records each.

A nested List in records with user_key (id) 1-10. The list is held in bin list_bin and has the following numeric, string, and List elements.



Numeric elements: user_key to user_key+4.
For example, for user_key=1: 1, 2, 3, 4, 5.
String elements: "s"+user_key to "s"+(user_key+4).
For example, for user_key=1: "s1", "s2", "s3", "s4", "s5"
Nested List element: holds numeric elements (10\*user_key)+1 to (10\*user_key)+5.

For example, for user_key=1: [11, 12, 13, 44, 15].
So, the value in list_bin for the record with user_key=1 looks like:

[ 1,"s1", 2, "s2", 3, "s3" 4, "s4", 5, "s5", [ 11, 12, 13, 14, 15 ] ].

![fig_CDTDM_1](./graphics/CDT_DM_1.png)

A nested Map in records with user_key (id) 101-110. The map is held in bin map_bin and has the following numeric, string, and List elements. Below, we have used i with a shorthand for i=user_key-100.


"oid": i.
For example, for i=1: 1.
"obj": a nested map with the following key-value pairs: {"attr1": (10\*i)+1, "attr2": "s"+(10\*i)+2, "subobj": {"attr3": (100\*i)+1, "attr4": (100\*i)+2}}.
For example, for i=1: {"attr1": 11, "attr2": "s12", "subobj": {"attr3": 101, "attr4": 102}}.
i: ["s"+i, "s"+i+1].

For example, for i=1: the key-value pair is 1: ["s1", "s2"].
So, the value in map_bin for the record with user_key=1 looks like:

{
  "oid": 1, 
  "obj": {"attr1": 11, "attr2": "s12", "subobj": {"attr3": 101, "attr4": 102}}, 
  1: ["s1", "s2"]
}

![fig_DM_2](./graphics/CDT_DM_2.png)


## Populate Test Data
Execute the cell below to populate the test data described above.

**We will insert 10 records with userkey=1 through 10 with ListBin, and  records with userkey = 101 thru 110 with MapBin.**

In [27]:
// Convenience function to execute a query using the input filter and print the results
void executeQueryAndPrintResults(Filter filter, String binName) {
    Statement stmt = new Statement();
    stmt.setNamespace(Namespace);
    stmt.setSetName(Set);
    stmt.setFilter(filter);
    stmt.setBinNames(binName);
    RecordSet rs = client.query(null, stmt);
    while (rs.next()) {
        Key key = rs.getKey();
        Record record = rs.getRecord();
        System.out.format("key=%s bins=%s\n", key.userKey, record.bins);
    }
    //System.out.println();
    rs.close();
}

WritePolicy wpolicy = new WritePolicy();
wpolicy.sendKey = true;

// create the list_bin with [i..i+4, "s"+i.."s"+4, [10*i+1..10*i+5]]
for (int user_key = 1; user_key <= 10; user_key++) {
    Key key = new Key(Namespace, Set, user_key);

    // append integer and string elements
    ArrayList<Value> list_val = new ArrayList<Value>();
    for (int n = user_key; n < user_key+5; n++) {
        list_val.add(Value.get(n));
        list_val.add(Value.get("s"+Integer.toString(n)));
    }

    // append string elements
    //for (int n=user_key; n < user_key+5; n++) {
    //    list_val.add(Value.get("s"+Integer.toString(n)));
    //}

    // append the nested list element
    ArrayList<Integer> list_int = new ArrayList<Integer>();
    for (int n=user_key*10+1; n < user_key*10+6; n++) {
        list_int.add(n);
    }
    ListValue list_int_val = new ListValue(list_int);

    // create the list bin
    Record record = client.operate(wpolicy, key,
                                    ListOperation.clear(ListBin), 
                                    ListOperation.appendItems(ListBin, list_val), 
                                    ListOperation.append(ListBin, list_int_val));
}

// create map bin with 
//    "oid": i,
//    "obj": {"attr1": 10*i+1, "attr2": "s"+10*+i+2, "subobj": {"attr3": 100*1+1, "attr4": 100*1+2}}, 
//    i: ["s"+i, "s"+i+1]


for (int user_key = 101; user_key <= 110; user_key++) {
    Integer i = user_key - 100;
    
    Key key = new Key(Namespace, Set, user_key);
    
    // construct obj map value
    HashMap <String, Value> obj = new HashMap <String, Value>();
    obj.put("attr1", Value.get(10*i+1));    
    obj.put("attr2", Value.get("s"+Integer.toString(10*i+2)));   
    HashMap <String, Integer> subobj = new HashMap <String, Integer>();
    subobj.put("attr3", 100*i+1);    
    subobj.put("attr4", 100*i+2);    
    obj.put("subobj", new MapValue(subobj));   

    // construct arr list value
    ArrayList<String> arr = new ArrayList<String>();
    arr.add("s"+Integer.toString(i*10+1));
    arr.add("s"+Integer.toString(i*10+2));
    
    // create the map in map_bin
    MapPolicy mPolicy = new MapPolicy(MapOrder.UNORDERED, MapWriteFlags.DEFAULT);
    Record record = client.operate(wpolicy, key,
                                MapOperation.clear(MapBin), 
                                MapOperation.put(mPolicy, MapBin, 
                                            Value.get("oid"), Value.get(i)),
                                MapOperation.put(mPolicy, MapBin, 
                                            Value.get("obj"), new MapValue(obj)),
                                MapOperation.put(mPolicy, MapBin, 
                                            Value.get(i), new ListValue(arr))
                                ); 
}

In [28]:
for (int user_key = 1; user_key <= 10; user_key++) {
    Key key = new Key(Namespace, Set, user_key);
    System.out.println(client.get(null, key));
}


(gen:1),(exp:0),(bins:(list_bin:[1, s1, 2, s2, 3, s3, 4, s4, 5, s5, [11, 12, 13, 14, 15]]))
(gen:1),(exp:0),(bins:(list_bin:[2, s2, 3, s3, 4, s4, 5, s5, 6, s6, [21, 22, 23, 24, 25]]))
(gen:1),(exp:0),(bins:(list_bin:[3, s3, 4, s4, 5, s5, 6, s6, 7, s7, [31, 32, 33, 34, 35]]))
(gen:1),(exp:0),(bins:(list_bin:[4, s4, 5, s5, 6, s6, 7, s7, 8, s8, [41, 42, 43, 44, 45]]))
(gen:1),(exp:0),(bins:(list_bin:[5, s5, 6, s6, 7, s7, 8, s8, 9, s9, [51, 52, 53, 54, 55]]))
(gen:1),(exp:0),(bins:(list_bin:[6, s6, 7, s7, 8, s8, 9, s9, 10, s10, [61, 62, 63, 64, 65]]))
(gen:1),(exp:0),(bins:(list_bin:[7, s7, 8, s8, 9, s9, 10, s10, 11, s11, [71, 72, 73, 74, 75]]))
(gen:1),(exp:0),(bins:(list_bin:[8, s8, 9, s9, 10, s10, 11, s11, 12, s12, [81, 82, 83, 84, 85]]))
(gen:1),(exp:0),(bins:(list_bin:[9, s9, 10, s10, 11, s11, 12, s12, 13, s13, [91, 92, 93, 94, 95]]))
(gen:1),(exp:0),(bins:(list_bin:[10, s10, 11, s11, 12, s12, 13, s13, 14, s14, [101, 102, 103, 104, 105]]))


In [29]:
for (int user_key = 101; user_key <= 110; user_key++) {
    Key key = new Key(Namespace, Set, user_key);
    System.out.println(client.get(null, key));
}


(gen:1),(exp:0),(bins:(map_bin:{1=[s11, s12], obj={attr2=s12, attr1=11, subobj={attr4=102, attr3=101}}, oid=1}))
(gen:1),(exp:0),(bins:(map_bin:{obj={attr2=s22, attr1=21, subobj={attr4=202, attr3=201}}, 2=[s21, s22], oid=2}))
(gen:1),(exp:0),(bins:(map_bin:{obj={attr2=s32, attr1=31, subobj={attr4=302, attr3=301}}, oid=3, 3=[s31, s32]}))
(gen:1),(exp:0),(bins:(map_bin:{4=[s41, s42], obj={attr2=s42, attr1=41, subobj={attr4=402, attr3=401}}, oid=4}))
(gen:1),(exp:0),(bins:(map_bin:{5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}))
(gen:1),(exp:0),(bins:(map_bin:{obj={attr2=s62, attr1=61, subobj={attr4=602, attr3=601}}, 6=[s61, s62], oid=6}))
(gen:1),(exp:0),(bins:(map_bin:{obj={attr2=s72, attr1=71, subobj={attr4=702, attr3=701}}, oid=7, 7=[s71, s72]}))
(gen:1),(exp:0),(bins:(map_bin:{8=[s81, s82], obj={attr2=s82, attr1=81, subobj={attr4=802, attr3=801}}, oid=8}))
(gen:1),(exp:0),(bins:(map_bin:{9=[s91, s92], obj={attr2=s92, attr1=91, subobj={attr4=902, attr3

## Non-Collection Indexes
We will illustrate how indexes on non-collection elements are created and used. We will start with results in mind, and then create the appropriate index, issue a query using the index, and show the results.

### Equality queries
Get records with a specific integer or string value at a specific index, rank, or key position of a List or a Map.

#### Records with a specific value at index X of a List or Map
##### The top level List in list_bin has has numeric value = 3 at List Index = 2.

![fig_EX_1](./graphics/CDT_EX_1.png)


In [30]:
// Create an index for element at position 2 of the top List in list_bin. (via the UI) 
// idx_list_bin_top_pos_2_num
// createIndex("idx_list_bin_top_pos_2_num", ListBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, CTX.listIndex(2));


// issue the query and print results
System.out.println("Records with the value 3 at position 2 of the top List in list_bin:");
Filter filter = Filter.equal(ListBin, 3, CTX.listIndex(2));
executeQueryAndPrintResults(filter, ListBin);

Records with the value 3 at position 2 of the top List in list_bin:
key=2 bins={list_bin=[2, s2, 3, s3, 4, s4, 5, s5, 6, s6, [21, 22, 23, 24, 25]]}


##### The top level Map in map_bin has numeric value 7 at map index = 2.


![fig_EX_2](./graphics/CDT_EX_2.png)

In [31]:
// Create an index for element at position 2 of the top Map in map_bin. Note, in the key ordered sequence, 
//   "oid" element is in position 2 after the numeric key and "obj".
// idx_map_bin_top_pos_2_num
//createIndex("idx_map_bin_top_pos_2_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, CTX.mapIndex(2));

// issue the query and print results
System.out.println("Records with the value 7 at position 2 of the top Map in map_bin:");
Filter filter = Filter.equal(MapBin, 7, CTX.mapIndex(2));
executeQueryAndPrintResults(filter, MapBin);

Records with the value 7 at position 2 of the top Map in map_bin:
key=107 bins={map_bin={obj={attr2=s72, attr1=71, subobj={attr4=702, attr3=701}}, oid=7, 7=[s71, s72]}}


### Records with a specific value at rank X of a List or Map
##### The nested List in list_bin has numeric value = 35 in rank -1 (highest value).

![fig_EX_3](./graphics/CDT_EX_3.png)

In [32]:
// Create an index for element at rank -1 of the nested List in list_bin. Note the nested List is at the 
//   last poistion (index -1) in the top List.
// idx_list_bin_nested_rnk_-1_num
// createIndex("idx_list_bin_nested_rnk_-1_num", ListBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
//            CTX.listIndex(-1), CTX.listRank(-1));

// issue the query and print results
System.out.println("Records with the value 35 in rank -1 (highest value) in the nested List in list_bin:");
Filter filter = Filter.equal(ListBin, 35, CTX.listIndex(-1), CTX.listRank(-1));
executeQueryAndPrintResults(filter, ListBin);

Records with the value 35 in rank -1 (highest value) in the nested List in list_bin:
key=3 bins={list_bin=[3, s3, 4, s4, 5, s5, 6, s6, 7, s7, [31, 32, 33, 34, 35]]}


##### The nested "subobj" Map in map_bin has numeric value = 901 in rank 0 (lowest value in all map values)
![fig_EX_4](./graphics/CDT_EX_4.png)

In [33]:
// Create an index for element at rank 0 (lowest value) in the nested "subobj" Map in map_bin. 
// idx_map_bin_subobj_rnk_0_num
// createIndex("idx_map_bin_subobj_rnk_0_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
//            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")), CTX.mapRank(0));

// issue the query and print results
System.out.println("Records with the value 901 in rank 0 (lowest value) in the nested \"subobj\" Map in map_bin:");
Filter filter = Filter.equal(MapBin, 901, 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")), CTX.mapRank(0));
executeQueryAndPrintResults(filter, MapBin);

Records with the value 901 in rank 0 (lowest value) in the nested "subobj" Map in map_bin:
key=109 bins={map_bin={9=[s91, s92], obj={attr2=s92, attr1=91, subobj={attr4=902, attr3=901}}, oid=9}}


#### Records with a specific value at key X of a Map
##### The top level Map in map_bin has numeric value = 8 at key "oid".
![fig_EX_5](./graphics/CDT_EX_5.png)

In [34]:
// Create an index for element with key "oid" in the top level Map in map_bin. 
// idx_map_bin_top_key_oid_num
//createIndex("idx_map_bin_top_key_oid_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
//            CTX.mapKey(Value.get("oid")));

// issue the query and print results
System.out.println("Records with the value 8 at key \"oid\" in the top level Map in map_bin:");
Filter filter = Filter.equal(MapBin, 8, 
            CTX.mapKey(Value.get("oid")));
executeQueryAndPrintResults(filter, MapBin);

Records with the value 8 at key "oid" in the top level Map in map_bin:
key=108 bins={map_bin={8=[s81, s82], obj={attr2=s82, attr1=81, subobj={attr4=802, attr3=801}}, oid=8}}


##### The nested "obj" Map in map_bin has string value = "s42" at map key = "attr2".
![fig_EX_6](./graphics/CDT_EX_6.png)

In [35]:
// Create an index for element with key "attr2" in the nested "obj" Map in map_bin. 
// idx_map_bin_obj_key_attr2_str
//createIndex("idx_map_bin_obj_key_attr2_str", MapBin, IndexType.STRING, IndexCollectionType.DEFAULT, 
//            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("attr2")));

// issue the query and print results
System.out.println("Records with the value \"s42\" at key \"attr2\" in the nested \"obj\" Map in map_bin:");
Filter filter = Filter.equal(MapBin, "s42", 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("attr2")));
executeQueryAndPrintResults(filter, MapBin);


Records with the value "s42" at key "attr2" in the nested "obj" Map in map_bin:
key=104 bins={map_bin={4=[s41, s42], obj={attr2=s42, attr1=41, subobj={attr4=402, attr3=401}}, oid=4}}


### Range Queries
Get records having an integer value within a range at a specific index, rank, or key position of a List or a Map. Range queries are supported on integer values only.

We will use the indexes defined above for the range queries too.

#### Records with value in s specific range at index X position of a List or Map
##### The top level List in list_bin numeric value is in range 3-5 at List Index = 2
![fig_EX_7](./graphics/CDT_EX_1.png)

In [ ]:
// issue the query and print results
// createIndex("idx_list_bin_top_pos_2_num", ListBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
//              CTX.listIndex(2)); 
// already created this SI

System.out.println("Records with value in range 3-5 at position 2 in the top level List in list_bin:");
Filter filter = Filter.range(ListBin, 3, 5, CTX.listIndex(2));
executeQueryAndPrintResults(filter, ListBin);

##### The top level Map in map_bin has numeric value in range 5 - 7 at map index = 2.
![fig_EX_8](./graphics/CDT_EX_8.png)

In [ ]:
// issue the query and print results
// createIndex("idx_map_bin_top_pos_2_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, CTX.mapIndex(2));  
// already created this SI
System.out.println("Records with value in range 5-7 at position 2 of the top Map in map_bin:");
Filter filter = Filter.range(MapBin, 5, 7, CTX.mapIndex(2));
executeQueryAndPrintResults(filter, MapBin);

#### Records with value in s specific range at rank X of a List or Map
##### The nested List in list_bin has numeric value in range 20 – 50 in rank -1 (highest value).
![fig_EX_9](./graphics/CDT_EX_3.png)

In [ ]:
// issue the query and print results
// createIndex("idx_list_bin_nested_rnk_-1_num", ListBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
//              CTX.listIndex(-1), CTX.listRank(-1));
// already created this SI
System.out.println("Records with value in range 20-50 in rank -1 (highest value) in the nested List in list_bin:");
Filter filter = Filter.range(ListBin, 20, 60, CTX.listIndex(-1), CTX.listRank(-1));
executeQueryAndPrintResults(filter, ListBin);

##### The nested "subobj" Map in map_bin has numeric value in range 500 – 800 in rank 0 (lowest value in all map values).
![fig_EX_10](./graphics/CDT_EX_4.png)

In [ ]:
// issue the query and print results
// createIndex("idx_map_bin_subobj_rnk_0_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
//              CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")), CTX.mapRank(0));  
// already created this SI

System.out.println("Records with value in range 500-800 in rank 0 (lowest value) in the nested \"subobj\" Map in map_bin:");
Filter filter = Filter.range(MapBin, 500, 801,
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")), CTX.mapRank(0));
executeQueryAndPrintResults(filter, MapBin);

#### Records with value in s specific range at key X of a Map

##### The top level Map in map_bin has numeric value in range 4 - 6 at key "oid".
![fig_EX_11](./graphics/CDT_EX_5.png)


In [ ]:
// issue the query and print results
// createIndex("idx_map_bin_top_key_oid_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, CTX.mapKey(Value.get("oid")));  
// already created this SI
System.out.println("Records with value in range 4-6 at key \"oid\" in the top level Map in map_bin:");
Filter filter = Filter.range(MapBin, 4, 6, CTX.mapKey(Value.get("oid")));
executeQueryAndPrintResults(filter, MapBin);

## Collection Indexes
Collection indexes are defined on List and Map elements with the collection type LIST (list values in the List), MAPKEYS (key values in the Map), or MAPVALUES (values in the Map). The indexes can be used for equality queries on integr and string elements, as well as range queries on integer elements. We will illustrate these variations below.

### Equality Queries
Get records with a specific integer or string value in a List or Map.

#### Records with a List or Map containing a specific value.

##### The nested List in map_bin at MapIndex = 0 contains "s91".
![fig_EX_12](./graphics/CDT_EX_7.png)

In [37]:
// Create an index for the nested List in map_bin. Note, in key-ordered sequence, the numeric key is the first one.
// idx_map_bin_i_list_str
//createIndex("idx_map_bin_i_list_str", MapBin, IndexType.STRING, IndexCollectionType.LIST, CTX.mapIndex(0));

// issue the query and print results
System.out.println("Records with the value \"s91\" in the nested list \"arr\" in map_bin:");
Filter filter = Filter.contains(MapBin, IndexCollectionType.LIST, "s91", CTX.mapIndex(0));
executeQueryAndPrintResults(filter, MapBin);

Records with the value "s91" in the nested list "arr" in map_bin:
key=109 bins={map_bin={9=[s91, s92], obj={attr2=s92, attr1=91, subobj={attr4=902, attr3=901}}, oid=9}}


##### The top level Map in map_bin contains a key whose numeric value = 5.
![fig_EX_13](./graphics/CDT_EX_9.png)

In [39]:
// Create an index for keys in top level Map in map_bin. 
// idx_map_bin_top_mapkeys_num
// createIndex("idx_map_bin_top_mapkeys_num", MapBin, IndexType.NUMERIC, IndexCollectionType.MAPKEYS);

// issue the query and print results
System.out.println("Records with key 5 in the top level Map in map_bin:");
Filter filter = Filter.contains(MapBin, IndexCollectionType.MAPKEYS, 5);
executeQueryAndPrintResults(filter, MapBin);

Records with key 5 in the top level Map in map_bin:
key=105 bins={map_bin={5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}}


##### The nested "obj" Map in map_bin contains a value "s12".
![fig_EX_14](./graphics/CDT_EX_10.png)

In [44]:
// Create an index for values in nested Map "obj" in map_bin. 
// idx_map_bin_obj_mapvals_str
//createIndex("idx_map_bin_obj_mapvals_str", MapBin, IndexType.STRING, IndexCollectionType.MAPVALUES, 
//            CTX.mapKey(Value.get("obj")));

// issue the query and print results
System.out.println("Records with the value \"s22\" in the nested Map \"obj\" in map_bin:");
Filter filter = Filter.contains(MapBin, IndexCollectionType.MAPVALUES, "s22", CTX.mapKey(Value.get("obj")));
executeQueryAndPrintResults(filter, MapBin);

Records with the value "s22" in the nested Map "obj" in map_bin:
key=102 bins={map_bin={obj={attr2=s22, attr1=21, subobj={attr4=202, attr3=201}}, 2=[s21, s22], oid=2}}


### Range Queries
Get records with a range of integer values in a List or Map.

#### Records with a List or Map containing an integer in a given range.

##### The nested list in list_bin has a value in the numeric value range 85-93.
![fig_EX_15](./graphics/CDT_EX_11.png)

In [51]:
// Create an index for the nested List in map_bin. Note, the nested list is the last element.
// idx_list_bin_nested_list_num
//createIndex("idx_list_bin_nested_list_num", ListBin, IndexType.NUMERIC, IndexCollectionType.LIST, 
//            CTX.listIndex(-1));

// issue the query and print results
System.out.println("Records with value in range 85-93 in the nested list in list_bin:");
Filter filter = Filter.range(ListBin, IndexCollectionType.LIST, 85, 93, CTX.listIndex(-1));
executeQueryAndPrintResults(filter, ListBin);

Records with value in range 85-93 in the nested list in list_bin:
key=9 bins={list_bin=[9, s9, 10, s10, 11, s11, 12, s12, 13, s13, [91, 92, 93, 94, 95]]}
key=8 bins={list_bin=[8, s8, 9, s9, 10, s10, 11, s11, 12, s12, [81, 82, 83, 84, 85]]}


##### The top level Map in map_bin has a MapKey in range 5-7.
![fig_EX_16](./graphics/CDT_EX_9.png)

In [ ]:
// use the index created earlier
// createIndex("idx_map_bin_top_mapkeys_num", MapBin, IndexType.NUMERIC, IndexCollectionType.MAPKEYS);  
// issue the query and print results
System.out.println("Records with key in range 5-7 in the top level Map in map_bin:");
Filter filter = Filter.range(MapBin, IndexCollectionType.MAPKEYS, 5, 7);
executeQueryAndPrintResults(filter, MapBin);

##### The nested Map "subobj" in map_bin has MapValue(s) in numeric value in range of 300 – 500.
![fig_EX_17](./graphics/CDT_EX_4.png)

In [ ]:
// Create an index for values in nested Map "subobj" in map_bin. 
// idx_map_bin_subobj_mapvals_num
//createIndex("idx_map_bin_subobj_mapvals_num", MapBin, IndexType.NUMERIC, IndexCollectionType.MAPVALUES, 
//            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")));

// issue the query and print results
System.out.println("Records with value in range 300-501 in the nested Map \"subobj\" in map_bin:");
Filter filter = Filter.range(MapBin, IndexCollectionType.MAPVALUES, 302, 501, 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")));
executeQueryAndPrintResults(filter, MapBin);

### Geo2DSphere Sindex Test
#### Truncate previous records and generate geoJSON test records

In [ ]:
Truncate tr = new Truncate();
tr.truncate(AEROSPIKE_CLOUD_NAMESPACE, "testset", 0);


#### GeoJSON Point, Polgon String generator - Helper functions


In [ ]:
//Generate Point Format GeoJSON string

//Point: order of data is longitude, latitude 

String generatePoint(double longitude, double latitude){
  return String.format("{\"type\": \"Point\", \"coordinates\": [%f, %f]}", longitude, latitude);
}

//Four corner polygon.  Specify in counterclockwise order
String generatePolygon(double ax, double ay, double bx, double by, double cx, double cy, double dx, double dy){
  return String.format(
  "{\"type\": \"Polygon\", \"coordinates\": [[[%f, %f], [%f, %f],[%f, %f],[%f, %f],[%f, %f]]]}", 
  ax, ay,
  bx, by,
  cx, cy,
  dx, dy,
  ax, ay
  );
}

String generateRectangle( double ax, double ay, double length, double height)
{
   length = length/2;
   height = height/2;
   return generatePolygon(ax-length, ay-height, ax+length, ay-height, ax+length, ay+height, ax-length,ay+height);
}
String generateAeroCircle(double x, double y, double r){
  return String.format(
  "{\"type\": \"AeroCircle\", \"coordinates\": [[%.8f, %.8f], %f] }", x, y, r);
}

System.out.println(generatePoint(0.0,0.0));
System.out.println(generateRectangle(0.0,0.0,4.0, 2.0));
System.out.println(generateAeroCircle(0.0,0.0,1.0));



#### GeoJSON Records with Point Bin


In [ ]:
//Insert a record using a class based construct
final String geoLocationBinName = "location";
final String geoRegionBinName = "region";
class geoUser {
  public void createLocationRecord(String ns, String setName, String user, double x, double y, boolean bCheck) {
    Key userKey = new Key(ns, setName, user); 
    String sLocPoint = generatePoint(x, y);
    Bin bLoc = new Bin(geoLocationBinName, Value.getAsGeoJSON(sLocPoint));    
    WritePolicy wPolicyGeo = new WritePolicy();
    wPolicyGeo.sendKey = true;
    client.put(wPolicyGeo, userKey, bLoc);
    if(bCheck){
      System.out.println(client.get(null, userKey));
    }   
  }
  
    public void createRegionRecord(String ns, String setName, String user, 
                                   double x, double y, double l, double h,boolean bCheck) {
    Key userKey = new Key(ns, setName, user); 
    String sRegion = generateRectangle(x, y, l, h);
    Bin bReg = new Bin(geoRegionBinName, Value.getAsGeoJSON(sRegion));    
    WritePolicy wPolicyGeo = new WritePolicy();
    wPolicyGeo.sendKey = true;
    client.put(wPolicyGeo, userKey, bReg);
    if(bCheck){
      System.out.println(client.get(null, userKey));
    }   
  }
}

void executeGeoQueryAndPrintResults(Filter filter, String binName) {
    Statement stmt = new Statement();
    stmt.setNamespace(Namespace);
    stmt.setSetName(Set);
    stmt.setFilter(filter);
    stmt.setBinNames(binName);
    RecordSet rs = client.query(null, stmt);
    while (rs.next()) {
        Key key = rs.getKey();
        Record record = rs.getRecord();
        System.out.format("key=%s bins=%s\n", key.userKey, record.bins);
    }
    //System.out.println();
    rs.close();
}

In [ ]:
geoUser ul = new geoUser();
ul.createLocationRecord(Namespace, Set, "user1", 0.0, 1.0, false);
ul.createLocationRecord(Namespace, Set, "user2", 1.0, 1.0, false);
ul.createLocationRecord(Namespace, Set, "user3", -1.0, 1.0, false);
ul.createLocationRecord(Namespace, Set, "user4", -1.0, -1.0, false);
ul.createLocationRecord(Namespace, Set, "user5", 0.0, -1.0, false);
ul.createLocationRecord(Namespace, Set, "user6", 1.0, -1.0, false);
ul.createLocationRecord(Namespace, Set, "user7", 1.0, 4.0, false);
ul.createLocationRecord(Namespace, Set, "user8", -1.0, 4.0, false);
ul.createLocationRecord(Namespace, Set, "user9", 0.0, 4.0, false);
ul.createLocationRecord(Namespace, Set, "user0", 0.0, -4.0, false);

//Add region bin also
ul.createRegionRecord(Namespace, Set, "user1", 0.0, 1.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user2", 1.0, 1.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user3", -1.0, 1.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user4", -1.0, -1.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user5", 0.0, -1.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user6", 1.0, -1.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user7", 1.0, 4.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user8", -1.0, 4.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user9", 0.0, 4.0, 0.25, 0.25, true);
ul.createRegionRecord(Namespace, Set, "user0", 0.0, -4.0, 0.25, 0.25, true);



#### GeoJSON SI Query. Find all Records with Point Bin where the Point lies within a geoJSON Rectangle


In [ ]:
String geoRect = generateRectangle(0.0,0.0,1.5,2.5);
System.out.println(geoRect);
Filter filter = Filter.geoContains(geoLocationBinName, geoRect);
executeGeoQueryAndPrintResults(filter, geoLocationBinName);

In [ ]:
String geoPoint = generatePoint(1.0, -1.0);
System.out.println(geoPoint);
Filter filter = Filter.geoContains(geoRegionBinName, geoPoint);
executeGeoQueryAndPrintResults(filter, geoRegionBinName);

#### GeoJSON SI Query. Find all Records with Point Bin where the Point lies within a geoJSON AeroCircle
**Note**
1 deg latitude is 69 miles or 110,400 meters. So choose radius in meters accordingly. All coordinates are in longitude and latitude degree units in GeoJSON.

In [ ]:
String geoCircle = generateAeroCircle(0.0,0.0,112000.0);
System.out.println(geoCircle);
//Filter filter = Filter.geoWithinRadius(geoLocationBinName, 0.0,0.0, 125000.0);
//Filter filter = Filter.geoWithinRegion(geoLocationBinName, IndexCollectionType.DEFAULT, geoCircle);
//Filter filter = Filter.geoWithinRegion(geoLocationBinName, geoCircle);
Filter filter = Filter.geoContains(geoLocationBinName, geoCircle);
executeGeoQueryAndPrintResults(filter, geoLocationBinName);

#### Test Sindex, at Scale

In [ ]:
   for(int i=0; i<100; i++){   
    Key userKey = new Key("aerospike_cloud", "testset", "user"+i); 
    Bin bName = new Bin("name", "user"+i);
    Bin bAge = new Bin("age", (2 + (i%100)));
    Bin bAge1 = new Bin("age1", (2 + (i%100)));
    Bin bAge2 = new Bin("age2", (2 + (i%100)));
    Bin bAge3 = new Bin("age3", (2 + (i%100)));
    Bin bAge4 = new Bin("age4", (2 + (i%100)));
    Bin bAge5 = new Bin("age5", (2 + (i%100)));
    Bin bAge6 = new Bin("age6", (2 + (i%100)));
    Bin bAge7 = new Bin("age7", (2 + (i%100)));
    Bin bAge8 = new Bin("age8", (2 + (i%100)));
    Bin bAge9 = new Bin("age9", (2 + (i%100)));
    Bin bAge10 = new Bin("age10", (2 + (i%100)));
    Bin bAge11 = new Bin("age11", (2 + (i%100)));
    Bin bAge12 = new Bin("age12", (2 + (i%100)));
    Bin bAge13 = new Bin("age13", (2 + (i%100)));
    Bin bAge14 = new Bin("age14", (2 + (i%100)));
    Bin bAge15 = new Bin("age15", (2 + (i%100)));
    Bin bAge16 = new Bin("age16", (2 + (i%100)));
    Bin bAge17 = new Bin("age17", (2 + (i%100)));
    Bin bAge18 = new Bin("age18", (2 + (i%100)));
    Bin bAge19 = new Bin("age19", (2 + (i%100)));
    
    WritePolicy wPolicy1 = new WritePolicy();
    wPolicy1.sendKey = true;
    client.put(wPolicy1, userKey, bName, bAge, bAge1, bAge2, bAge3,bAge4,bAge5,bAge6,bAge7,bAge8,bAge9,bAge10,bAge11,bAge12,bAge13,bAge14,bAge15,bAge16,bAge17,bAge18,bAge19);
    }

In [ ]:
//client.close();